# Overview

This notebook provides the code to recreate the simulations included in the "Fair Feature Importance Scores for Interpreting Tree-based Methods and Surrogates". The simulate_data function simulates data as specified by the input parameters. We provide decision a decision tree classifier, boosting classifier, and random forest classifier example on a linear scenario. We also provide a decision tree regressor, boosting regressor, and random forest regressor example on a linear scenario. 

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import math
from sklearn.tree import DecisionTreeRegressor
from scipy.special import expit
from scipy.stats import pearsonr
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from FairFIS import fis_tree, fis_forest, fis_boosting
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, RandomForestRegressor
from FairFIS import util
import random

In [ ]:
def simulate_data(nrow, ncol, alphas, betas, p, seed, model_type, classification = True):

    """
    Simulates biased data.
    Inputs:
      * nrow - the number of rows in the output dataset.
      * ncol - the number of non-protected covariates X_i in the output dataset.
      * alphas - numpy array of scalars, where alpha_i controls the effect of
        protected attribute z on covariate X_i through the relationship
        X_i ~ Normal(alpha_i*z, 1).
      * betas - numpy array of scalars, where beta_i controls the effect of
        covariate X_i on the binary outcome y; can be thought of as the regression
        coefficients in the logistic regression scenario.
      * p - the probability of success (aka value 1) for our protected attribute;
        should be less than 0.5.
      * model_type - Options are 'linear', 'nonlinear', 'nonlinear_int'. These correspond to 
        a linear model, a nonlinear additive model, and a nonlinear additive model with 
        pairwise interactions.
      * classification = True (default) will return a classification dataset, 
        while classifcation = False will return a regression dataset.
    Returns:
      * X matrix, y vector, z vector
    """

    # the following two assertions are to check that the input parameters
    # make sense - we should have one value of alpha and beta for each covariate
    assert ncol == len(alphas)
    assert ncol == len(betas)
    random.seed(seed)
    betas = np.reshape(betas, (len(betas),1))
    z = np.random.binomial(1, p, size=nrow)
    X = np.zeros((nrow, ncol))

    if model_type == 'linear':
        for i in range(ncol):
            X[:,i] =  alphas[i]*z + np.random.normal(loc = 0 , scale = 0.1, size = nrow)
        y_prob = X@betas + np.random.normal(loc = 0 , scale = 0.01, size = (nrow,1))
        
    if model_type == 'nonlinear':
        for i in range(ncol):
            X[:,i] =  alphas[i]*z + np.random.normal(loc = 0 , scale = 0.1, size = nrow)
        y_prob = np.sin(X)@betas + np.random.normal(loc = 0 , scale = 0.01, size = (nrow,1))
        
    if model_type == 'nonlinear_int':
        for i in range(ncol):
            X[:,i] =  alphas[i]*z + np.random.normal(loc = 0 , scale = 0.1, size = nrow)
            X[:,0] = X[:,0]*X[:,1]
            X[:,3] = X[:,3]*X[:,4]
            X[:,6] = X[:,6]*X[:,7]
        y_prob = np.sin(X)@betas + np.random.normal(loc = 0 , scale = 0.01, size = (nrow,1))
    
    if classification == True:
        y_prob = expit(y_prob)
        y = (y_prob >= .5).astype(int)
    else:
        y = y_prob
  
    return X,y,z

# Classification Example

In [ ]:
#Classification Simulation Setup
iterations = 10
seed_vec = list(range(iterations))
nrow = 1000
ncol = 12
a = 2
b = 1
alphas = np.array([a,a,a,a,a,a,0,0,0,0,0,0])
betas = np.array([b,b,b,0,0,0,b,b,b,0,0,0])
p = 0.2

### Decision Tree Classifier

In [ ]:
dp_mat = np.empty([iterations,ncol])
eo_mat = np.empty([iterations,ncol])
acc_mat = np.empty([iterations,ncol])

for i, s in enumerate(seed_vec):
  X,y,z = simulate_data(nrow,ncol,alphas, betas,p,s,'linear')
  clf = DecisionTreeClassifier(max_depth = 8)
  clf.fit(X,y)
  
  #FairFIS
  f_forest = fis_tree(clf,X,y,z,0, triangle = False)
  f_forest.calculate_fairness_importance_score()
  eo_mat[i] = f_forest._fairness_importance_score_eqop_root
  dp_mat[i] = f_forest._fairness_importance_score_dp_root
  acc_mat[i] = clf.feature_importances_

eo_mean = np.mean(eo_mat, axis = 0)
dp_mean = np.mean(dp_mat, axis = 0)
acc_mean = np.mean(acc_mat, axis = 0)

In [ ]:
#Display results for decision tree classifier
features = [1,2,3,4,5,6,7,8,9,10,11,12]
data = {'Feature': features,
    'DP': dp_mean,
         'EO': eo_mean,
             'ACC': acc_mean}
  
# Create DataFrame
tree_scores = pd.DataFrame(data)

### Boosting Classifier

In [ ]:
dp_mat = np.empty([iterations,ncol])
eo_mat = np.empty([iterations,ncol])
acc_mat = np.empty([iterations,ncol])

for i, s in enumerate(seed_vec):
  X,y,z = simulate_data(nrow,ncol,alphas, betas,p,s,'linear')
  clf = GradientBoostingClassifier(n_estimators=100, max_depth=5, max_features='auto')
  clf.fit(X,y)
  
  #FairFIS
  f_forest = fis_boosting(clf,X,y,z,0, triangle = False)
  f_forest.calculate_fairness_importance_score()
  
  dp_mat[i] = f_forest._fairness_importance_score_dp_root
  eo_mat[i] = f_forest._fairness_importance_score_eqop_root
  acc_mat[i] = f_forest.fitted_clf.feature_importances_



dp_mean = np.mean(dp_mat, axis = 0)
eo_mean = np.mean(eo_mat, axis = 0)
acc_mean = np.mean(acc_mat, axis = 0)

In [ ]:
#Display results for boosting classifier
features = [1,2,3,4,5,6,7,8,9,10,11,12]
data = {'Feature': features,
    'DP': dp_mean,
         'EO': eo_mean,
             'ACC': acc_mean}
  
# Create DataFrame
boosting_scores = pd.DataFrame(data)

### Random Forest Classifier

In [ ]:
dp_mat = np.empty([iterations,ncol])
eo_mat = np.empty([iterations,ncol])
acc_mat = np.empty([iterations,ncol])

for i, s in enumerate(seed_vec):
  X,y,z = simulate_data(nrow,ncol,alphas, betas,p,s,'linear')
  clf = RandomForestClassifier(n_estimators=100,n_jobs=-2)
  clf.fit(X,y)
  
  #Our approach
  f_forest = fis_forest(clf,X,y,z,0, triangle = False)
  f_forest.calculate_fairness_importance_score()
  
  dp_mat[i] = f_forest._fairness_importance_score_dp_root
  eo_mat[i] = f_forest._fairness_importance_score_eqop_root
  acc_mat[i] = f_forest.fitted_clf.feature_importances_


eo_mean = np.mean(eo_mat, axis = 0)
dp_mean = np.mean(dp_mat, axis = 0)
acc_mean = np.mean(acc_mat, axis = 0)

In [ ]:
features = [1,2,3,4,5,6,7,8,9,10,11,12]
data = {'Feature': features,
    'DP': dp_mean,
         'EO': eo_mean,
             'ACC': acc_mean}
  
# Create DataFrame
rf_scores = pd.DataFrame(data)

# Regression Example

In [ ]:
#Regression Simulation Setup
iterations = 10
seed_vec = list(range(iterations))
nrow = 1000
ncol = 12
a = 0.5
b = 5
alphas = np.array([a,a,a,a,a,a,0,0,0,0,0,0])
betas = np.array([b,b,b,0,0,0,b,b,b,0,0,0])
p = 0.2

### Decision Tree Regressor

In [ ]:
dp_mat = np.empty([iterations,ncol])
eo_mat = np.empty([iterations,ncol])
acc_mat = np.empty([iterations,ncol])

for i, s in enumerate(seed_vec):
  X,y,z = simulate_data(nrow,ncol,alphas, betas,p,s,'linear', classification = False)
  clf = DecisionTreeRegressor(max_depth = 8)
  clf.fit(X,y)
  
  #Our approach
  f_forest = fis_tree(clf,X,y,z,0, regression=True)
  f_forest.calculate_fairness_importance_score()
  
  dp_mat[i] = f_forest._fairness_importance_score_dp_root
  acc_mat[i] = clf.feature_importances_



dp_mean = np.mean(dp_mat, axis = 0)
acc_mean = np.mean(acc_mat, axis = 0)

In [ ]:
features = [1,2,3,4,5,6,7,8,9,10,11,12]
data = {'Feature': features,
    'DP': dp_mean,
         'EO': eo_mean,
             'ACC': acc_mean}
  
# Create DataFrame
tree_scores = pd.DataFrame(data)

### Boosting Regressor

In [ ]:
dp_mat = np.empty([iterations,ncol])
eo_mat = np.empty([iterations,ncol])
acc_mat = np.empty([iterations,ncol])

for i, s in enumerate(seed_vec):
  X,y,z = simulate_data(nrow,ncol,alphas, betas,p,s,'linear', classification = False)
  clf = GradientBoostingRegressor(n_estimators=100, max_depth=5, max_features='auto')
  clf.fit(X,y)
  
  #Our approach
  f_forest = fis_boosting(clf,X,y,z,0, regression=True)
  f_forest.calculate_fairness_importance_score()
  
  dp_mat[i] = f_forest._fairness_importance_score_dp_root
  acc_mat[i] = f_forest.fitted_clf.feature_importances_



dp_mean = np.mean(dp_mat, axis = 0)
acc_mean = np.mean(acc_mat, axis = 0)

In [ ]:
#Display results for boosting regressor
features = [1,2,3,4,5,6,7,8,9,10,11,12]
data = {'Feature': features,
    'DP': dp_mean,
         'EO': eo_mean,
             'ACC': acc_mean}
  
# Create DataFrame
boosting_scores = pd.DataFrame(data)

### Random Forest Regressor

In [ ]:
dp_mat = np.empty([iterations,ncol])
eo_mat = np.empty([iterations,ncol])
acc_mat = np.empty([iterations,ncol])

for i, s in enumerate(seed_vec):
  X,y,z = simulate_data(nrow,ncol,alphas, betas,p,s,'linear', classification = False)
  clf = RandomForestRegressor(n_estimators=100,n_jobs=-2)
  clf.fit(X,y)
  
  #Our approach
  f_forest = fis_forest(clf,X,y,z,0, regression=True)
  f_forest.calculate_fairness_importance_score()
  
  dp_mat[i] = f_forest._fairness_importance_score_dp_root
  acc_mat[i] = f_forest.fitted_clf.feature_importances_



dp_mean = np.mean(dp_mat, axis = 0)
acc_mean = np.mean(acc_mat, axis = 0)

In [ ]:
#Display results for regression regressor
features = [1,2,3,4,5,6,7,8,9,10,11,12]
data = {'Feature': features,
    'DP': dp_mean,
         'EO': eo_mean,
             'ACC': acc_mean}
  
# Create DataFrame
regression_scores = pd.DataFrame(data)